In this project you are trying out for an online marketplace company. The evaluation cover 3 important aspects of the daily job there that you need to know in order to get the position. 

The name of the database you will be working with is: data-analyst-web-store



Step 1:

Queries to know the data:

1. How many customers are there and what is the gender ratio?

2. How many items are there? Do you see any phenomena? (notice that item_id can be different)

3. What is the total amount of orders?

4. How many users were registered in July – December 2019 on the YouTube campaign?

5. How many users from the US bought the book Rich Dad? What is their share out of everyone who bought this book? And what is their share out of all US sales?

6. What is the most ordered item by quantity and what is the item with the highest total revenue? 

7. What is the share of total revenue, orders and users registration for each country, And how this ratio changes from 2019 to 2020 (show results for both years)

8. What is the total of shipments for each category in the data?

9. What is the average of the first order of each customer?

10. Does the membership split equally between gender? Do we get the same result per different countries in the data? Check only for English speaking countries (US, CA and EN)

11. What is the most successful campaign in terms of registrations? And what is the most successful campaign in terms of total revenue brought in?


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2

In [3]:
db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',       #host URL      
                      'port': 6432,              # connection port             
                      'db': 'data-analyst-web-store'}          # the name of the database

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [4]:
def queryResult(q):
    return pd.io.sql.read_sql(q, con = engine)

In [5]:
query1 = '''
            SELECT gender,COUNT(DISTINCT customer_id)
            FROM customer_info
            GROUP BY gender
         '''
display(queryResult(query1))

,gender,count
0,Female,241
1,Male,218




1. How many customers are there and what is the gender ratio?



In [6]:

query1 = '''
            select
                sum(case when "gender" = 'Male' then 1 else 0 end)/count(*)::float as male_ratio,
                sum(case when "gender" = 'Female' then 1 else 0 end)/count(*)::float as female_ratio,
                sum(customer_id) as total
            from 
                customer_info
         '''
display(queryResult(query1))

,male_ratio,female_ratio,total
0,0.477083,0.522917,117859


2. How many items are there? Do you see any phenomena? (notice that item_id can be different)



3. What is the total amount of orders?

In [7]:
query1 = '''
            select sum(amount) as total_orders
            from order_details
         '''
display(queryResult(query1))

,total_orders
0,3180


4. How many users were registered in July – December 2019 on the YouTube campaign?

In [9]:
query1 = '''
            
         '''
display(queryResult(query1))

ProgrammingError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: 
            
         ]
(Background on this error at: http://sqlalche.me/e/14/f405)

5. How many users from the US bought the book Rich Dad? What is their share out of everyone who bought this book? And what is their share out of all US sales?

In [8]:
query1 = '''
            select sum(amount) as sum_US_total_sales_Rich_Dad
            from 
                items
                left join 
                        (
                        order_details
                        left join 
                                ( 
                                orders
                                left join 
                                        ( 
                                        customer_info
                                        left join countries on countries.country_id = customer_info.country_id
                                        ) as cst_info on cst_info.customer_id = orders.customer_id
                                ) 
                            as cst_orders on cst_orders.order_id = order_details.order_id
                        ) as cst_order_details on cst_order_details.item_id = items.item_id
                where "item" = 'Rich Dad' and "symbol" = 'US'
         '''
# where "item" = 'Rich Dad'
display(queryResult(query1))

,sum_us_total_sales_rich_dad
0,2


6. What is the most ordered item by quantity and what is the item with the highest total revenue? 

In [11]:
query1 = '''
            
         '''
display(queryResult(query1))

ProgrammingError: (psycopg2.ProgrammingError) can't execute an empty query
[SQL: 
            
         ]
(Background on this error at: http://sqlalche.me/e/14/f405)


7. What is the share of total revenue, orders and users registration for each country, And how this ratio changes from 2019 to 2020 (show results for both years)


In [9]:
query1 = '''
            select
                country,
                year,
                revenue / sum(revenue) over (partition by year) *100 as revenue_ratio,
                order_cnt / sum(order_cnt) over (partition by year) *100 as ordr_ratio,
                customer_cnt / sum(customer_cnt) over (partition by year) *100 as registration_ratio
            from
                (select 
                        country,
                        sum(price * amount) as revenue,
                        count(distinct cst_order_items.order_id) as order_cnt,
                        count(distinct cst_order_items.customer_id) as customer_cnt,
                        extract(year from date) :: int as year
                from 
                        registration
                        left join (
                                select i.* , cst_order_details.order_id, cst_order_details.country, cst_order_details.customer_id, cst_order_details.amount, cst_order_details.date
                                from 
                                items i
                                left join 
                                        (
                                        select od.* , cst_orders.country, cst_orders.customer_id, cst_orders.date
                                        from order_details od
                                        left join 
                                                (
                                                select orders.* , cst_inf_ct.country
                                                from orders 
                                                left join 
                                                        ( 
                                                        select cst_info.*, countries.country
                                                        from customer_info cst_info
                                                        left join countries on countries.country_id = cst_info.country_id
                                                        ) as cst_inf_ct on cst_inf_ct.customer_id = orders.customer_id
                                                ) as cst_orders on cst_orders.order_id = od.order_id
                                        ) as cst_order_details on cst_order_details.item_id = i.item_id
                                ) as cst_order_items on cst_order_items.customer_id = registration.customer_id
                where extract(year from date) > 2018 and extract(year from date) < 2021
                group by country, year) as final_table
        order by country
         '''


display(queryResult(query1))

,country,year,revenue_ratio,ordr_ratio,registration_ratio
0,Belguim,2019,1.394121,3.797468,3.759398
1,Belguim,2020,6.668262,6.562500,7.027027
2,Canada,2019,17.109629,14.556962,15.789474
3,Canada,2020,9.139087,13.906250,13.513514
4,England,2020,8.048886,5.781250,5.135135
5,England,2019,3.904886,7.594937,6.766917
6,France,2020,11.208516,10.625000,10.000000
7,France,2019,16.037476,10.126582,10.526316
8,Germany,2019,7.728985,6.962025,7.518797
9,Germany,2020,11.706142,8.750000,8.648649



8. What is the total of shipments for each category in the data?


In [ ]:
query1 = '''
            
         '''
display(queryResult(query1))


9. What is the average of the first order of each customer?


In [ ]:
query1 = '''
            
         '''
display(queryResult(query1))


10. Does the membership split equally between gender? Do we get the same result per different countries in the data? Check only for English speaking countries (US, CA and EN)


In [ ]:
query1 = '''
            
         '''
display(queryResult(query1))


11. What is the most successful campaign in terms of registrations? And what is the most successful campaign in terms of total revenue brought in?


In [80]:
query1 = '''
            select
                campaign_id,
                camp_rev,
                mem_cnt 
            from
                (select 
                    campaign_id,
                    sum(items.price * cst_order_details.amount) as camp_rev,
                    count(case when "membership" = 'yes' then 1 else 0 end) as mem_cnt
                from 
                    items
                    left join 
                            (
                            select 
                            
                                order_details.item_id,
                                order_details.amount, 
                                order_details.order_id, 
                                cst_orders.campaign_id,
                                cst_orders.membership
                            from
                                order_details
                                left join 
                                        ( 
                                        select orders.order_id, orders.customer_id, cst_info.campaign_id,  cst_info.membership
                                        from 
                                            orders
                                            left join 
                                                    ( 
                                                    select registration.campaign_id, registration.customer_id, customer_info.membership
                                                    from
                                                        customer_info
                                                        left join registration on registration.customer_id = customer_info.customer_id
                                                    ) as cst_info on cst_info.customer_id = orders.customer_id
                                        ) as cst_orders on cst_orders.order_id = order_details.order_id
                            ) as cst_order_details on cst_order_details.item_id = items.item_id
                    group by campaign_id) final_Table
                order by camp_rev desc, mem_cnt desc
                limit 2

         '''
                            #     count(order_details.item_id) as item_id_cnt,
                            # sum(order_details.amount) as sum_amount, 
                            # count(order_details.order_id) as order_cnt , 
                            # cst_orders.campaign_id, 
                            # sum(case when "membership" = 'yes' then 1 else 0 end) as sum_membership
display(queryResult(query1))

,campaign_id,camp_rev,mem_cnt
0,2,64520.39,477
1,3,58216.37,354
